In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import poisson
import math
import plotly.graph_objects as go
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<script src="https://unpkg.com/@lottiefiles/lottie-player@latest/dist/lottie-player.js"></script>
<lottie-player src="https://assets1.lottiefiles.com/packages/lf20_bD8Yze.json"  background="transparent"  speed="1"  style="width: 300px; height: 300px;"  loop controls autoplay></lottie-player>

<img src="https://s4.gifyu.com/images/soccer1.gif" width="200" height="200">

In [ ]:
calcio = pd.read_csv('/kaggle/input/sports-betting/calcio_italiano_risultati_scomesse.csv')
calcio = calcio.loc[calcio.week > 5] #Отрезаем ранние игры без накопленной статистики
calcio_columns = list(calcio.columns) 
calcio_columns[9] = 'team1_draw' #Исправляем название
calcio.columns = calcio_columns

Датасет старый, в нём заложена маржа букмекера 10%, что на сегодня многова-то, переведём на маржу 5% и отдельно расчитаем вероятности без маржи для каждого исхода.

<img src="https://s4.gifyu.com/images/pesce1.gif" width="200" height="200">

In [ ]:
calcio['K1'] = round(calcio.P1 * (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2) / 1.05, 2)
calcio['KD'] = round(calcio.PD * (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2) / 1.05, 2)
calcio['K2'] = round(calcio.P2 * (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2) / 1.05, 2)
calcio['R1'] = 1 / calcio.P1 / (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2)
calcio['RD'] = 1 / calcio.PD / (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2)
calcio['R2'] = 1 / calcio.P2 / (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2)

Отрежем матчи с очень высокими вероятностями и с очень низкими по расчетам букмекеров.

<img src="https://s4.gifyu.com/images/animation_200_kndprinu.gif" width="200" height="200">

In [ ]:
print(len(calcio))
calcio = calcio.loc[(calcio.R1 > 0.15) & (calcio.R1 <0.85)]
calcio = calcio.iloc[:-10,:]
print(len(calcio))

Как рассчитать переоценку или недооценку коэффициента букмекера для создания модели предсказания?
1. Нужно фактический исход матча и взять его результат как самый вероятный исход. т.е. силу атаки команды коэффициенты как количество мячей забитых командой в матче.
2. Рассчитать по Пуассону вероятности других исходов
3. Суммировать вероятности исходов для расчёта вероятностей победы, ничьей и проигрыша.
4. Перевести вероятности в коэффициенты и вычесть из коэфициенты букмекера, отрицательные значения будут означать переоценённые ставки, положительные недооценённые.
* к нулевым результатам нужно добавить небольшой коэффициенты, чтобы получить распределение.

<img src="https://s4.gifyu.com/images/st.cou0yDkTwFaE.gif" width="400" height="400">

In [ ]:
calcio['home_goals_fact'] = calcio.team1_goal #фактический результат
calcio['away_goals_fact'] = calcio.team2_goal
calcio.loc[calcio['home_goals_fact'] == 0, 'home_goals_fact'] = 0.25 #небольшой коэффициент к 0, 
calcio.loc[calcio['away_goals_fact'] == 0, 'away_goals_fact'] = 0.175 #чтобы получить распределение Пуассона
#Для каждой команды расчитаем вероятность забить n -мячей
calcio['f_home_0'] = poisson.pmf(0.25, mu=calcio.home_goals_fact, loc=0) 
calcio['f_away_0'] = poisson.pmf(0.175, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_1'] = poisson.pmf(1, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_1'] = poisson.pmf(1, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_2'] = poisson.pmf(2, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_2'] = poisson.pmf(2, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_3'] = poisson.pmf(3, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_3'] = poisson.pmf(3, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_4'] = poisson.pmf(4, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_4'] = poisson.pmf(4, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_5'] = poisson.pmf(5, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_5'] = poisson.pmf(5, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_6'] = poisson.pmf(6, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_6'] = poisson.pmf(6, mu=calcio.away_goals_fact, loc=0)

Список исходов для победы, ничьей и проигрыша.

<img src="https://s4.gifyu.com/images/liverpool.gif" width="200" height="200">

In [ ]:
home_win_results = [
                    (1, 0), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), 
                    (4, 0), (4, 1), (4, 2), (4, 3), 
                    (5, 0), (5, 1), (5, 2), (5, 3), (5, 4),
                    (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5)
                    ]
draw_results = [
                    (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6)
                    ]
away_win_results = [
                    (0, 1), (0, 2), (1, 2), (0, 3), (1, 3), (2, 3), 
                    (0, 4), (1, 4), (2, 4), (3, 4), 
                    (0, 5), (1, 5), (2, 5), (3, 5), (4, 5),
                    (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6)
                    ]

Суммируем вероятности по исходам

<img src="https://s4.gifyu.com/images/st.coZxBVPNwf8B.gif" width="400" height="400">

In [ ]:
calcio['f_home_win'] = 0
for res in home_win_results:
    calcio['f_home_win'] = calcio['f_home_win'] + calcio['f_home_' + str(res[0])]*calcio['f_away_' + str(res[1])]

calcio['f_draw'] = 0
for res in draw_results:
    calcio['f_draw'] = calcio['f_draw'] + calcio['f_home_' + str(res[0])]*calcio['f_away_' + str(res[1])]

calcio['f_away_win'] = 0
for res in away_win_results:
    calcio['f_away_win'] = calcio['f_away_win'] + calcio['f_home_' + str(res[0])]*calcio['f_away_' + str(res[1])]

Переводим в коэффициенты

In [ ]:
calcio['f_prob_sum'] = calcio['f_home_win'] + calcio['f_draw'] + calcio['f_away_win']
calcio['fRP1'] = calcio.f_home_win / calcio.f_prob_sum
calcio['fRPD'] = calcio.f_draw / calcio.f_prob_sum
calcio['fRP2'] = calcio.f_away_win / calcio.f_prob_sum

Считаем переоценку/недооценку:

<img src="https://s4.gifyu.com/images/animation_500_kndd0zof.gif" width="200" height="200">

In [ ]:
calcio['coeff_fact'] = calcio.R1-calcio.fRP1
calcio['coeff_fact_sign'] = 1
calcio.loc[calcio.coeff_fact <= 0, 'coeff_fact_sign'] = -1

In [ ]:
calcio.head()

In [ ]:
calcio_hover = calcio[[
        'year', 'week', 'home', 'away', 'team1_curr_stand', 'team2_curr_stand', 
        'team1_curr_goals_home', 'team2_curr_concs_away', 
        'team2_curr_goals_away', 'team1_curr_concs_home'
        ]].values

In [ ]:
fig = go.Figure(data=go.Scatter(
    x = calcio.team1_goal - calcio.team2_goal,
    y = calcio.coeff_fact,
    #z = calcio.R1,
    hovertemplate =
    '<b>Year</b>: %{customdata[0]}'+
    '<br><b>Week</b>: %{customdata[1]}' +
    '<br><b>Home</b>: %{customdata[2]}' +
    '<br><b>Away</b>: %{customdata[3]}' +
    '<br><b>Stand Home</b>: %{customdata[4]}' +
    '<br><b>Stand Away</b>: %{customdata[5]}' +
    '<br><b>Goals Home</b>: %{customdata[6]:.2f}' +
    '<br><b>Conc Home</b>: %{customdata[7]:.2f}' +
    '<br><b>Goals Away</b>: %{customdata[8]:.2f}' +
    '<br><b>Conc Away</b>: %{customdata[9]:.2f}',
    customdata = calcio_hover,
    mode='markers',
    marker=dict(
        size = 5,
        color=calcio.team1_win,
        colorscale='Viridis',
        line_width=1
    )
))
fig.update_layout( 
                dict(
                    margin=dict(l=0, r=0, b=0, t=0),                            
                    title = (f'Calcio Seria A 2004 - 2013 PCA for Home Team'),
                    autosize=False,
                    width=800,
                    height=800,
                    xaxis = dict(title = 'Goal diifference Home - Away'),
                    yaxis = dict(title = 'Estimation error after match'),
                    #zaxis = dict(title = 'Probability')
                    #yaxis2 = yaxis2
                        )
                )
fig.show()

Верхний график: 
X - счёт
Y - переоценка/недооценка
1. Слева вверху: недооценка и проигрыш хозяев
2. Справа вверху: недооценка и выигрыш хозяев
3. Справа внизу: переоценка и выигрыш хозяев(то что нам нужно)
4. Слева внизу: переоценка и проигрыш хозяев(то что нам совсем не нужно)

In [ ]:
fig = go.Figure(data=go.Scatter(
    x = calcio.R1,
    y = calcio.coeff_fact,
    #z = calcio.team1_goal - calcio.team2_goal,
    hovertemplate =
    '<b>Year</b>: %{customdata[0]}'+
    '<br><b>Week</b>: %{customdata[1]}' +
    '<br><b>Home</b>: %{customdata[2]}' +
    '<br><b>Away</b>: %{customdata[3]}' +
    '<br><b>Stand Home</b>: %{customdata[4]}' +
    '<br><b>Stand Away</b>: %{customdata[5]}' +
    '<br><b>Goals Home</b>: %{customdata[6]:.2f}' +
    '<br><b>Conc Home</b>: %{customdata[7]:.2f}' +
    '<br><b>Goals Away</b>: %{customdata[8]:.2f}' +
    '<br><b>Conc Away</b>: %{customdata[9]:.2f}',
    customdata = calcio_hover,
    mode='markers',
    marker=dict(
        size = 5,
        color=calcio.team1_win,
        colorscale='Viridis',
        line_width=1
    )
))
fig.update_layout( 
                dict(
                    margin=dict(l=0, r=0, b=0, t=0),                            
                    title = (f'Calcio Seria A 2004 - 2013 PCA for Home Team'),
                    autosize=False,
                    width=800,
                    height=800,
                    xaxis = dict(title = 'Probability before match'),
                    yaxis = dict(title = 'Estimation error after match'),
                    #zaxis = dict(title = 'Probability')
                    #yaxis2 = yaxis2
                        )
                )
fig.show()

Верхний график: 
X - Вероятность букмекера
Y - переоценка/недооценка

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x = calcio.R1,
    y = calcio.coeff_fact,
    z = calcio.team1_goal - calcio.team2_goal,
    hovertemplate =
    '<b>Year</b>: %{customdata[0]}'+
    '<br><b>Week</b>: %{customdata[1]}' +
    '<br><b>Home</b>: %{customdata[2]}' +
    '<br><b>Away</b>: %{customdata[3]}' +
    '<br><b>Stand Home</b>: %{customdata[4]}' +
    '<br><b>Stand Away</b>: %{customdata[5]}' +
    '<br><b>Goals Home</b>: %{customdata[6]:.2f}' +
    '<br><b>Conc Home</b>: %{customdata[7]:.2f}' +
    '<br><b>Goals Away</b>: %{customdata[8]:.2f}' +
    '<br><b>Conc Away</b>: %{customdata[9]:.2f}',
    customdata = calcio_hover,
    mode='markers',
    marker=dict(
        size = 5,
        color=calcio.team1_win,
        colorscale='Viridis',
        line_width=1
    )
))
fig.update_layout( 
                dict(
                    margin=dict(l=0, r=0, b=0, t=0),                            
                    title = (f'Calcio Seria A 2004 - 2013 PCA for Home Team'),
                    autosize=False,
                    width=800,
                    height=800,
                    xaxis = dict(title = 'Probability before match'),
                    yaxis = dict(title = 'Estimation error after match'),
                    #zaxis = dict(title = 'Probability')
                    #yaxis2 = yaxis2
                        )
                )
fig.show()